In [126]:
import os
import sys
import glob
import time
import datetime
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import random
import scipy as sp

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from functools import partial
from multiprocessing.pool import Pool
from time import time

In [231]:
file_zero = pd.read_csv("Songs/songs0.csv")
file_zero["fid"] = 0
file_one = pd.read_csv("Songs/songs1.csv")
file_one["fid"] = 1
complete_playlists = pd.DataFrame(np.concatenate([file_zero, file_one]), columns = file_zero.columns)

In [232]:
complete_playlists

,pid,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,fid
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,0
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0
5,0,5,Usher,spotify:track:0XUfyU2QviPAs6bxSpXYG4,spotify:artist:23zg3TcAtWQy7J6upgbUnj,Yeah!,spotify:album:0vO0b1AvY49CPQyVisJLj0,250373,Confessions,0
6,0,6,Usher,spotify:track:68vgtRHr7iZHpzGpon6Jlo,spotify:artist:23zg3TcAtWQy7J6upgbUnj,My Boo,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,223440,Confessions,0
7,0,7,The Pussycat Dolls,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,Buttons,spotify:album:5x8e8UcCeOgrOzSnDGuPye,225560,PCD,0
8,0,8,Destiny's Child,spotify:track:7H6ev70Weq6DdpZyyTmUXk,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,Say My Name,spotify:album:283NWqNsCA9GwVHrJk59CG,271333,The Writing's On The Wall,0
9,0,9,OutKast,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,Hey Ya! - Radio Mix / Club Mix,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,235213,Speakerboxxx/The Love Below,0


In [233]:
all_unique_songs = complete_playlists['track_name'].unique()
complete_playlists.set_index(["fid", "pid"], inplace=True)

In [234]:
complete_playlists

pos            artist_name                             track_uri  \
fid pid                                                                    
0   0     0          Missy Elliott  spotify:track:0UaMYEvWZi0ZqiDOoHU3YI   
    0     1         Britney Spears  spotify:track:6I9VzXrHxO9rA9A5euc8Ak   
    0     2                Beyoncé  spotify:track:0WqIKmW4BTrj3eJFmnCKMv   
    0     3      Justin Timberlake  spotify:track:1AWQoqb9bSvzTjaLralEkT   
    0     4                 Shaggy  spotify:track:1lzr43nnXAijIGYnCT8M8H   
    0     5                  Usher  spotify:track:0XUfyU2QviPAs6bxSpXYG4   
    0     6                  Usher  spotify:track:68vgtRHr7iZHpzGpon6Jlo   
    0     7     The Pussycat Dolls  spotify:track:3BxWKCI06eQ5Od8TY2JBeA   
    0     8        Destiny's Child  spotify:track:7H6ev70Weq6DdpZyyTmUXk   
    0     9                OutKast  spotify:track:2PpruBYCo4H7WOBJ7Q2EwM   
    0    10          Nelly Furtado  spotify:track:2gam98EZKrF9XuOkU13ApN   
    0    11        Jesse McCartney  spotify:track:4Y45aqo9QMa57rDsAJv40A   
    0    12        Jesse McCartney  spotify:track:1HwpWwa6bnqqRhK8agG4RS   
    0    13        Jesse McCartney  spotify:track:20ORwCJusz4KS2PbTPVNKo   
    0    14                 Cassie  spotify:track:7k6IzwMGpxnRghE7YosnXT   
    0    15                Omarion  spotify:track:1Bv0Yl01xBDZD4OQP93fyl   
    0    16          Avril Lavigne  spotify:track:4omisSlTk6Dsq2iQD7MA07   
    0    17            Chris Brown  spotify:track:7xYnUQigPoIDAMPVK79NEq   
    0    18                Beyoncé  spotify:track:6d8A5sAx9TfdeseDvfWNHd   
    0    19        Destiny's Child  spotify:track:4pmc2AxSEq6g7hPVlJCPyP   
    0    20            Sheryl Crow  spotify:track:215JYyyUnrJ98NK3KEwu6d   
    0    21    The Black Eyed Peas  spotify:track:0uqPG793dkDDN7sCUJJIVC   
    0    22       Bowling For Soup  spotify:track:19Js5ypV6JKn4DMExHQbGc   
    0    23         The Click Five  spotify:track:1JURww012QnWAw0zZXi6Aa   
    0    24            Chris Brown  spotify:track:7DFnq8FYhHMCylykf6ZCxA   
    0    25         Jonas Brothers  spotify:track:1TfAhjzRBWzYZ8IdUV3igl   
    0    26               Lil Mama  spotify:track:1Y4ZdPOOgCUhBcKZOrUFiS   
    0    27                Cascada  spotify:track:6MjljecHzHelUDismyKkba   
    0    28           Jason Derulo  spotify:track:67T6l4q3zVjC5nZZPXByU8   
    0    29                  Ne-Yo  spotify:track:34ceTg8ChN5HjrqiIYCn9Q   
...      ..                    ...                                   ...   
1   998  15               Bon Jovi  spotify:track:0J6mQxEZnlRt9ymzFntA6z   
    998  16                  Train  spotify:track:6mghCOaaSvrke0z1EUVUIf   
    998  17                  Train  spotify:track:4ocbH5ZJqajZIpUrqRUoue   
    998  18      Five For Fighting  spotify:track:5gBQ3x6u7DCvMW1QoP3CpG   
    998  19               The Fray  spotify:track:5fVZC9GiM4e8vu99W0Xf6J   
    998  20               The Fray  spotify:track:4IoYz8XqqdowINzfRrFnhi   
    998  21      Simon & Garfunkel  spotify:track:0iOZM63lendWRTTeKhZBSC   
    998  22                Journey  spotify:track:4bHsxqR3GMrXTxEPLuK5ue   
    998  23                Journey  spotify:track:71SvEDmsOwIWw1IozsZoMA   
    998  24              Tom Petty  spotify:track:5tVA6TkbaAH9QMITTQRrNv   
    998  25  Red Hot Chili Peppers  spotify:track:3d9DChrdc6BOeFsbrZ3Is0   
    998  26  Red Hot Chili Peppers  spotify:track:10Nmj3JCNoMeBQ87uw5j8k   
    998  27           Lukas Graham  spotify:track:7675gjlUZzneYiMrQ9Inx8   
    998  28        Louis Armstrong  spotify:track:29U7stRjqHU6rMiS8BfaI9   
    999   0              blink-182  spotify:track:2wMqxbWlfIIaVzpYtFG4HY   
    999   1              blink-182  spotify:track:1e1krgHCTUGDhhIgsRVHJ0   
    999   2      Breaking Benjamin  spotify:track:4BJyt25nburVwbnESDeIc7   
    999   3              Crossfade  spotify:track:6ZxrUCaynLgKSUudACOTwj   
    999   4                   Enya  spotify:track:6FLwmdmW77N1Pxb1aWsZmO   
    999   5                 Filter  spotify:track:2c5wml

In [131]:
#complete_playlists.loc[(354,557)]["track_name"]

In [235]:
unique_playlist_ids = np.unique(complete_playlists.index.values)
np.random.shuffle(unique_playlist_ids)
train_idx_cutoff = int(0.8 * unique_playlist_ids.shape[0])
train_playlist_ids = unique_playlist_ids[0:train_idx_cutoff]
test_playlist_ids = unique_playlist_ids[train_idx_cutoff:unique_playlist_ids.shape[0]]

In [236]:
def createSparseMatrix(train_playlist_ids):    
    rows_list = []
    num_processed = 0
    for ids in train_playlist_ids:
        playlist_songs = complete_playlists.loc[ids]["track_name"]
        ones_idx = np.where(np.isin(all_unique_songs, playlist_songs))[0]
        row = np.zeros(all_unique_songs.shape)
        row[ones_idx] = 1 / playlist_songs.shape[0]
        rows_list.append(row)
        num_processed += 1
    train_sparse = np.vstack(rows_list)
    return train_sparse

In [237]:
train_sparse = createSparseMatrix(train_playlist_ids)

In [238]:
compressed_sparse = sp.sparse.csr_matrix(train_sparse)
model_knn_compressed = NearestNeighbors(metric='cosine', 
                                        algorithm='brute', 
                                        n_neighbors=20, n_jobs=-1).fit(train_sparse)

In [239]:
rows_list = []
val_list = []
# For each playlist in test playlists
for ids in test_playlist_ids:
    test_playlist = complete_playlists.loc[ids]
    
    # Split into test songs and validation songs
    # Could be better
    #np.random.shuffle(test_playlists)
    idx = np.random.choice(np.arange(len(test_playlist)), int(0.8 * len(test_playlist)), replace=False)
    test_idx = np.where(np.in1d(np.arange(len(test_playlist)), idx, invert=True))[0]
    songs_to_use = test_playlist["track_name"].values[idx]
    songs_to_evaluate = test_playlist.iloc[test_idx]
    val_list.append(songs_to_evaluate)
    
    # Make the matrix for test playlist
    ones_idx = np.where(np.in1d(all_unique_songs, songs_to_use))[0]
    row = np.zeros(all_unique_songs.shape)
    row[ones_idx] = 1 # / songs_to_use.shape[0]
    rows_list.append(row)
    
test_sparse = np.vstack(rows_list)
compressed_test_sparse = sp.sparse.csr_matrix(np.vstack(rows_list))
val_df = pd.concat(val_list)
test_indices = test_playlist_ids

In [240]:
model_knn_compressed.kneighbors(test_sparse)

(array([[0.9139337 , 0.92531488, 0.9272607 , ..., 0.9503096 , 0.95111763,
         0.95150714],
        [0.94942783, 0.95149287, 0.95243485, ..., 0.97288369, 0.9730516 ,
         0.97498436],
        [0.89057156, 0.90925575, 0.91130314, ..., 0.9674872 , 0.96785878,
         0.96785878],
        ...,
        [0.90829685, 0.91777828, 0.91783134, ..., 0.95453371, 0.95453371,
         0.95503062],
        [0.80146337, 0.84969784, 0.85826079, ..., 0.94292182, 0.94340835,
         0.9439888 ],
        [0.9595774 , 0.96597931, 0.9676238 , ..., 0.9841807 , 0.98442436,
         0.98462448]]), array([[ 422,  807,  399, ...,  808,  103,  265],
        [1029, 1058,  190, ...,  431, 1049,  239],
        [ 392,  214,  577, ..., 1488, 1351, 1389],
        ...,
        [1445,  226,  897, ...,   44,  267, 1470],
        [1089,  697, 1495, ..., 1399,  810,  183],
        [ 193, 1493,  977, ..., 1394, 1522,  880]]))

In [241]:
def getSongPredictions(X, model, indices, candidate_playlists, top_n, col_names):
    distances, neighbors = model.kneighbors(X)
    predictions_list = []
    for i in range(X.shape[0]):
        best_candidates = neighbors[i][distances[i] != 1]
        best_distances = distances[i][distances[i] != 1]
        if best_candidates.size != 0:
            # get the place in candidate_playlists with the lowest distance
            candidate_idx = indices[best_candidates]
        else:
            # TODO: Change
            predictions_list.append(pd.DataFrame(columns = candidate_playlists.columns))
            continue
            
        candidates_df = candidate_playlists.loc[list(map(tuple, candidate_idx))]
        
        # transform the best_distances into a form that can be concatenated
        lengths = candidates_df.groupby(level=[0,1])["pos"].count().values
        

        candidates_df["distance"] = np.repeat(best_distances, lengths)
        
        # want only the songs that are not in the playlist
        not_in_train_df = candidates_df.iloc[np.where(np.in1d(candidates_df["track_name"].values, 
                                                      col_names[X[i].toarray()[0] != 0], invert=True))[0]]
        
        # drop duplicates
        no_duplicates_df = not_in_train_df.drop_duplicates("track_name")
        if no_duplicates_df.shape[0] < top_n:
            predictions_list.append(no_duplicates_df)
        else:
            predictions_list.append(no_duplicates_df.iloc[0:top_n])
    return predictions_list

In [242]:
predictions_list = getSongPredictions(compressed_test_sparse, model_knn_compressed, 
                                      train_playlist_ids, complete_playlists, 500, all_unique_songs)

# Evaluate

In [243]:
clicks = []
for i in range(compressed_test_sparse.shape[0]):
    predictions = predictions_list[i]
    idx = test_indices[i]
    val_songs = val_df.loc[tuple(idx)]
    positions_matching = np.where(np.in1d(predictions["track_name"].values, val_songs['track_name']))[0]
    # CLICKS: min pos / 10
    if positions_matching.size == 0:
        clicks.append(50)
    else:
        clicks.append(math.ceil(positions_matching[0] / 10))

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  """


In [247]:
np.mean(clicks)

14.2675

In [245]:
complete_playlists.loc[test_playlist_ids[4]]

pos        artist_name                             track_uri  \
fid pid                                                                
0   178   0              Drake  spotify:track:4kNvYhyl8R6m1vykVkcuBu   
    178   1          Rob $tone  spotify:track:5uDASfU19gDxSjW8cnCaBp   
    178   2              Aminé  spotify:track:5hTpBe8h35rJ67eAWHQsJx   
    178   3        Joey Bada$$  spotify:track:4ntsD33Pgsgk8rUyZLLiUV   
    178   4               DRAM  spotify:track:7yyRTcZmCiyzzJlNzGC9Ol   
    178   5              Drake  spotify:track:433P7tDcIAi6NLnf4Sh6tI   
    178   6               Russ  spotify:track:57evRskKe47TvzG96SLLTM   
    178   7               KYLE  spotify:track:4L4DOYJFB3QiYIssnMDCrd   
    178   8  Chance The Rapper  spotify:track:0v9Wz8o0BT8DU38R4ddjeH   
    178   9              Drake  spotify:track:3NxAG2ni1lLa8RKL6a0INc   
    178  10        Kodak Black  spotify:track:0BU7nIcCZxYSE0hYcc5oZz   
    178  11        Post Malone  spotify:track:5yuShbu70mtHXY0yLzCQLQ   
    178  12              Drake  spotify:track:343YBumqHu19cGoGARUTsd   
    178  13         Lil Yachty  spotify:track:40yJGQyEaBbIkXpwMFkXip   
    178  14          Fetty Wap  spotify:track:2d8JP84HNLKhmd6IYOoupQ   
    178  15        Post Malone  spotify:track:3a1lNhkSLSkpJE4MSHpDu9   
    178  16          Mike Stud  spotify:track:5tsI3xxDHDgzHhn30LTQNz   
    178  17               KYLE  spotify:track:2EEeOnHehOozLq4aS0n6SL   
    178  18          Lil Wayne  spotify:track:4dASQiO1Eoo3RJvt74FtXB   
    178  19       Rae Sremmurd  spotify:track:6mapJIPnQ23RTAevUoE0DL   
    178  20       Denzel Curry  spotify:track:6R0GRYk2vs2XuBVemYK5YZ   
    178  21       Rae Sremmurd  spotify:track:4scpF6J5uMBvoh6sFB7EL1   
    178  22              Drake  spotify:track:11KJSRSgaDxqydKYiD2Jew   
    178  23         The Weeknd  spotify:track:5aAx2yezTd8zXrkmtKl66Z   
    178  24           Big Sean  spotify:track:0SGkqnVQo9KPytSri1H6cF   
    178  25             Eminem  spotify:track:6sDQ4uiWw9OdVrCXFLSlZt   
    178  26       Lil Uzi Vert  spotify:track:2ANLarE8yHVsLWW21nj79M   
    178  27         Kanye West  spotify:track:4KW1lqgSr8TKrvBII0Brf8   
    178  28       Rae Sremmurd  spotify:track:2RUuKIHFHjk41E7YsJ9HNP   
    178  29          Mike Stud  spotify:track:59oP06RoBps6bvbvSQ1Zee   
...      ..                ...                                   ...   
    178  40        Kodak Black  spotify:track:4Q3N4Ct4zCuIHuZ65E3BD4   
    178  41         Polo Frost  spotify:track:7BQ1LYgw8XGhnUFY851OVg   
    178  42              Drake  spotify:track:7hDc8b7IXETo14hHIHdnhd   
    178  43       Lil Uzi Vert  spotify:track:7GX5flRQZVHRAGd6B4TmDO   
    178  44       Travis Scott  spotify:track:6gBFPUFcJLzWGx4lenP6h2   
    178  45             Future  spotify:track:0VgkVdmE4gld66l8iyGjgx   
    178  46              Migos  spotify:track:76Y0gxtTxN0FyDCYh5qYQj   
    178  47              Drake  spotify:track:2bjwRfXMk4uRgOD9IBYl9h   
    178  48               Russ  spotify:track:5dM52n0cmm1Kt8abAP1l9Z   
    178  49        Mir Fontane  spotify:track:6toI90xEX8PssRHv9FSVag   
    178  50        Nebu Kiniza  spotify:track:6JjEVlMkfHWMeYavkEYzNO   
    178  51           The Game  spotify:track:24b20V609aI0WJ8LyxR8qx   
    178  52  Chance The Rapper  spotify:track:6m9qPYXmhge2QhBLfFKnVF   
    178  53          DJ Khaled  spotify:track:3DXncPQOG4VBw3QHh3S817   
    178  54  Chance The Rapper  spotify:track:5IdQEHgtmj9th3OkfQKhf8   
    178  55  Chance The Rapper  spotify:track:6VndlbaMjAscnEeqrVkMN2   
    178  56  Chance The Rapper  spotify:track:3ZLyt2ndLFBh148XRYjYYZ   
    178  57  Chance The Rapper  spotify:track:0jx8zY5JQsS4YEQcfkoc5C   
    178  58             Khalid  spotify:track:152lZdxL1OR0ZMW6KquMif   
    178  59     Kendrick Lamar  spotify:track:7KXjTSCq5nL1LoYtL7XAwS   
    178  60              Drake  spotify:track:5mPSyjLatqB00IkPqRlbTE   
    178  61                KSI  spotify:track:1mhdEO1cBTccQ3tknmM6ng   
    178  62         Logan Paul  spotify:track:51

In [246]:
val_df.loc[test_playlist_ids[4]]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


pos     artist_name                             track_uri  \
fid pid                                                             
0   178  14       Fetty Wap  spotify:track:2d8JP84HNLKhmd6IYOoupQ   
    178  22           Drake  spotify:track:11KJSRSgaDxqydKYiD2Jew   
    178  26    Lil Uzi Vert  spotify:track:2ANLarE8yHVsLWW21nj79M   
    178  41      Polo Frost  spotify:track:7BQ1LYgw8XGhnUFY851OVg   
    178  42           Drake  spotify:track:7hDc8b7IXETo14hHIHdnhd   
    178  43    Lil Uzi Vert  spotify:track:7GX5flRQZVHRAGd6B4TmDO   
    178  48            Russ  spotify:track:5dM52n0cmm1Kt8abAP1l9Z   
    178  50     Nebu Kiniza  spotify:track:6JjEVlMkfHWMeYavkEYzNO   
    178  58          Khalid  spotify:track:152lZdxL1OR0ZMW6KquMif   
    178  59  Kendrick Lamar  spotify:track:7KXjTSCq5nL1LoYtL7XAwS   
    178  60           Drake  spotify:track:5mPSyjLatqB00IkPqRlbTE   
    178  63             KSI  spotify:track:5ZT1oRyGphljlYQOyE7ELt   
    178  64       21 Savage  spotify:track:2fQrGHiQOvpL9UgPvtYy6G   
    178  69    Travis Scott  spotify:track:5cgEadiGKjtTP1Q6NJqVP5   

                                    artist_uri     track_name  \
fid pid                                                         
0   178  spotify:artist:6PXS4YHDkKvl1wkIl4V8DL     Trap Queen   
    178  spotify:artist:3TVXtAsR1Inumwj472S9r4       Too Good   
    178  spotify:artist:4O15NlyKLIASxsJ0PrXPfz  You Was Right   
    178  spotify:artist:6HTHu9OnzdrUp3NryLbxAk      Best Ever   
    178  spotify:artist:3TVXtAsR1Inumwj472S9r4   Passionfruit   
    178  spotify:artist:4O15NlyKLIASxsJ0PrXPfz  XO TOUR Llif3   
    178  spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS       Too Many   
    178  spotify:artist:5lCY3tqdQxbeg5igSlObaT      Gassed Up   
    178  spotify:artist:6LuN9FCkKOj5PcnpouEgny       Location   
    178  spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg        HUMBLE.   
    178  spotify:artist:3TVXtAsR1Inumwj472S9r4      Pop Style   
    178  spotify:artist:1nzgtKYFckznkcVMR3Gg4z    Kilimanjaro   
    178  spotify:artist:1URnnhqYAYcrqrcwql10ft   Bank Account   
    178  spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY         A-Team   

                                    album_uri duration_ms  \
fid pid                                                     
0   178  spotify:album:2gKQvajkEEaDtkqJ8FJ4uw      222093   
    178  spotify:album:3hARKC8cinq3mZLLAEaBh9      263373   
    178  spotify:album:7mgdTKTCdfnLoa1HXHvLYM      163944   
    178  spotify:album:4DO7ZaebQrFaJNaCpOlwXX      226721   
    178  spotify:album:7Ix0FS4f1lK42C3rix5rHg      298940   
    178  spotify:album:733e1ZfktLSwj96X5rsMeE      182706   
    178  spotify:album:5s4FdPwnFJTFmdNPtVlLsp      161071   
    178  spotify:album:6YCcGxz3l1shgr3F7XbNee      193911   
    178  spotify:album:6kf46HbnYCZzP6rjvQHYzg      219080   
    178  spotify:album:4eLPsYPBmXABThSJ821sqY      177000   
    178  spotify:album:40GMAhriYJRO1rsY4YdrZb      212946   
    178  spotify:album:4HHgTXeepZwV8dVJWI8d9t      229227   
    178  spotify:album:4QUNWTw2FsIz722xjUjFJw      220306   
    178  spotify:album:4Q9gIbDuQiVOexTyDxx9V0      176640   

                         album_name  
fid pid                              
0   178                   Fetty Wap  
    178                       Views  
    178  Lil Uzi Vert Vs. The World  
    178                   Best Ever  
    178                   More Life  
    178               Luv Is Rage 2  
    178                    Too Many  
    178                   Gassed Up  
    178               American Teen  
    178                       DAMN.  
    178                       Views  
    178                     Keep Up  
    178                  Issa Album  
    178                      A-Team

In [207]:
complete_playlists[complete_playlists["track_uri"] == "spotify:track:11pfbduq7fbQxmXH03OKo7"]

pos artist_name                             track_uri  \
fid pid                                                          
0   362   19   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   
    432   43   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   
    433    8   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   
    648   30   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   
    919   42   Sam Smith  spotify:track:11pfbduq7fbQxmXH03OKo7   

                                    artist_uri       track_name  \
fid pid                                                           
0   362  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   
    432  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   
    433  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   
    648  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   
    919  spotify:artist:2wY79sveU1sp5g7SokKOiI  Not In That Way   

                                    album_uri  duration_ms          album_name  
fid pid                                                                         
0   362  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour  
    432  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour  
    433  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour  
    648  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour  
    919  spotify:album:2Jg7JZ0ZXOGje1bkq7CVgK       172452  In The Lonely Hour

In [177]:
predictions_list[4]

pos             artist_name                             track_uri  \
fid pid                                                                      
0   607    0      Corinne Bailey Rae  spotify:track:2l1BYvzTOKFKDLAXeQc3SE   
    607    1              Chet Faker  spotify:track:2xD7RvKFJl52xyds3wQFXQ   
    607    2          The Civil Wars  spotify:track:4zzi2eD2cEPpQ3a307mPPj   
    607    3                  Jungle  spotify:track:3g86N0SqyfxEVNMHlmW3RE   
    607    4      Family of the Year  spotify:track:3MkcVmqLSxHTkTFgczctbQ   
    607    5               Ben Folds  spotify:track:4hVDEHZgzxSdYkQl732TvW   
    607    6          Gary Clark Jr.  spotify:track:6dT6nC1IxBPBF71t4IFm5G   
    607    7           Chris Cornell  spotify:track:4iWm8M2rSRAkmZYFGs12Fx   
    607    8                Coldplay  spotify:track:7D0RhFcb3CrfPuTJ0obrod   
    607    9            When In Rome  spotify:track:48p5E25cFPanxuwCTmTpuL   
    607   10              Us The Duo  spotify:track:6KzL8s8GdRbd5Mal8yLz6R   
    607   11         Yeah Yeah Yeahs  spotify:track:2hxV5yGCr1nIp5bQBxXL13   
    607   12      Corinne Bailey Rae  spotify:track:7aFQa52bLiHwb3BXPbmKdZ   
    607   13        Sleeping At Last  spotify:track:0dHF8t7OIhgPele5WRUfWG   
    607   14                  Hozier  spotify:track:2tKOIBLJ2j384tnunmmKi7   
    607   15         Freedy Johnston  spotify:track:5aNAOwr2QoRbvyRG7ZVrhf   
    607   16         Yeah Yeah Yeahs  spotify:track:3zvHOVC4jikl6jORSr6OLE   
    607   17                    Beck  spotify:track:3arVrdpOPMgLZOztBr2jM6   
    607   18          Regina Spektor  spotify:track:3FvxvqQmMCZLsIzNvFQqhz   
    607   19            Matt and Kim  spotify:track:57i3M29DWoo7RDk0Tf0LZG   
    607   20        Butthole Surfers  spotify:track:1ng36571Iyov4HBxUClySn   
    607   21                   Adele  spotify:track:7rPLZ8Krm6CZIbraFUlnWZ   
    607   22               Cat Power  spotify:track:6Pi8GUiL7VqbPbU8IhHxyP   
    607   23                  Jungle  spotify:track:6Pu1G8OV5m38PkLQHyZfPp   
    607   24     Trampled By Turtles  spotify:track:06gmYLiwfegMk3yHx26vjB   
    607   25           Black Sabbath  spotify:track:3b7wNR330iKhql9dEdbNci   
    607   26     Death Cab for Cutie  spotify:track:4mCF3EBgGPSqmEm205KBAV   
    607   27              Aloe Blacc  spotify:track:1FsH6OZCKLWbBgqj8JcCvO   
    607   28  James Vincent McMorrow  spotify:track:2hSdHOOBLXNkh7zmmgoAn3   
    607   29            Jack Johnson  spotify:track:12jjuxN1gxlm29cqL5M6MW   
...      ...                     ...                                   ...   
    13     2              Rob Thomas  spotify:track:1Zx5Y46u3v32jRRIWtzQKN   
    13     3           Jeffrey James  spotify:track:3dFJn0eFwXGZipJMpDgFsW   
    13     5        Jasmine Thompson  spotify:track:2PeQP3wjJ0ogwbUr1X6tU1   
    13     6         Gabrielle Aplin  spotify:track:7pq5vONxrAy8T773nhkiW3   
    13     7         Christina Perri  spotify:track:6lanRgr6wXibZr8KgzXxBl   
    13     8      Molly Kate Kestner  spotify:track:66ZEsTGwfUCcslLyIDCxzR   
    13     9         Katy McAllister  spotify:track:0RJmnZ23Usaa07LPUe4mGb   
    13    10          Joseph Vincent  spotify:track:07ceQy7UhpNDA2WhEOJgPp   
    13    11            James Arthur  spotify:track:3ZdCBj1cbossYy1O40ZmLD   
    13    12                 Ruth B.  spotify:track:5hwh37sTi84MVhCBMWzhGE   
    272    1              Bruno Mars  spotify:track:6SKwQghsR8AISlxhcwyA9R   
    272    2               Sam Smith  spotify:track:0fioLzGM8ngbD1w6fMmm45   
    272    3              Bruno Mars  spotify:track:7BqBn9nzAq8spo5e7cZ0dJ   
    272    4                   B.o.B  spotify:track:59dLtGBS26x7kc0rHbaPrq   
    272    5              Katy Perry  spotify:track:0ktV2JoOsoTGURzKaZnjJL   
    272    6                Maroon 5  spotify:track:494OU6M7NOf4ICYb4zWCf5   
    272    7              John Mayer  spotify:track:0QTCTu0CXv4X1JEE4gNpGv   
    272    8           X Ambassadors  spotify:track:46GGxd8TVRt6FjUBfCavVT   
    272    9                  M

# MULTIPROCESSING

In [74]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.getLogger('requests').setLevel(logging.CRITICAL)
logger = logging.getLogger(__name__)


def main():
    ts = time()
    with Pool(4) as p:
        p.map(createSparseMatrix, playlist_list)
    logging.info('Took %s seconds', time() - ts)


if __name__ == '__main__':
    main()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
2019-12-09 15:46:04,484 - root - INFO - Took 1.4989278316497803 seconds
